<a href="https://colab.research.google.com/github/saanikagupta/End-to-End-Autonomous-Mobile-Robot-Navigation/blob/master/Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

# Google Drive

## Mounting Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!pwd
# !ls 'drive/My Drive/'

/content


# Reproducible results

In [0]:
import numpy as np
import tensorflow as tf
import random as rn

In [0]:
import os
os.environ['PYTHONHASHSEED'] = '0'

In [0]:
# Setting the seed for NumPy generated random numbers
np.random.seed(27)

# Setting the seed for Python random numbers
rn.seed(1254)

# Setting the seed for Tensorflow random numbers
tf.set_random_seed(89)
from keras import backend as K

# Force tensorflow to use a single thread
session_conf = tf.ConfigProto(intra_op_parallelism_threads = 1, inter_op_parallelism_threads = 1)
sess = tf.Session(graph = tf.get_default_graph(), config = session_conf)
K.set_session(sess)

# Model

In [0]:
tf.logging.set_verbosity(tf.logging.ERROR)
from keras import layers, models, utils

In [0]:
def MyModel(input_shape_X, input_shape_Goal):
    
    # Three class classification
    num_classes = 3

    # Define the input as a tensor with shape 
    X_input = layers.Input(shape = input_shape_X)
    Goal_input = layers.Input(shape = input_shape_Goal)
    
    X = layers.Conv1D(64, 7, strides = 3, padding = 'valid')(X_input)
    X = layers.BatchNormalization()(X)
    X = layers.ReLU()(X)
    
    X = layers.MaxPooling1D(3)(X)
    
    # For shortcut connection
    X_shortcut = X
    
    X = layers.Conv1D(64, 3, strides = 1, padding = 'same')(X)
    X = layers.BatchNormalization()(X)
    X = layers.ReLU()(X)
    
    X = layers.Conv1D(64, 3, strides = 1, padding = 'same')(X)
    X = layers.BatchNormalization()(X)
    
    # For shortcut connection
    X_shortcut_2 = X 
    
    # Add shortcut value to main path, and pass it through a RELU activation
    X = layers.Add()([X_shortcut, X])
    X = layers.ReLU()(X)
    
    X = layers.Conv1D(64, 3, strides = 1, padding = 'same')(X)
    X = layers.BatchNormalization()(X)
    X = layers.ReLU()(X)
    
    X = layers.Conv1D(64, 3, strides = 1, padding = 'same')(X)
    X = layers.BatchNormalization()(X)
    
    # Add shortcut value to main path, and pass it through a RELU activation
    X = layers.Add()([X_shortcut_2, X])
    X = layers.ReLU()(X)
    
    X = layers.AveragePooling1D(pool_size = 3, strides = None, padding = 'valid')(X)
    
    # Concatinating the output from CNN and the input goal information
    X = layers.Flatten()(X)
    X = layers.merge.Concatenate()([X, Goal_input])
    
    # FC layers
    X = layers.Dense(1024, activation = 'relu')(X)    
    X = layers.Dropout(rate = 0.75, seed = 0)(X) # Applying Dropout
    X = layers.Dense(1024, activation = 'relu')(X)
    X = layers.Dropout(rate = 0.65, seed = 0)(X) # Applying Dropout
    X = layers.Dense(512, activation = 'linear')(X)
    X = layers.Dropout(rate = 0.5, seed = 0)(X) # Applying Dropout
    X = layers.Dense(num_classes, activation = 'softmax')(X)
    
    # Create model
    model = models.Model(name = 'MyModel', input = [X_input, Goal_input], output = X)

    return model

# Training

In [0]:
import pandas as pd
# import glob
# from random import shuffle
import time
import shutil

In [47]:
# Importing dataset
shutil.move("drive/My Drive/balanced_normalized_dataset.zip", "balanced_normalized_dataset.zip")
!unzip 'balanced_normalized_dataset.zip'
shutil.move("balanced_normalized_dataset.zip", "drive/My Drive/balanced_normalized_dataset.zip")

Archive:  balanced_normalized_dataset.zip
replace balanced_normalized_dataset/las_thetag_dg_w_e1g1_4.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

'drive/My Drive/balanced_normalized_dataset.zip'

In [0]:
# csv_list = glob.glob('balanced_normalized_dataset/*.csv')
# # print(csv_list)
# shuffle(csv_list)
# print(csv_list)

In [0]:
# list_len = len(csv_list)
# print(list_len)

In [0]:
# train_len = int(list_len * 0.75)

In [0]:
# train_csv_list = csv_list[: train_len]
# test_csv_list = csv_list[train_len: ]

In [0]:
train_csv_list = ['balanced_normalized_dataset/las_thetag_dg_w_e10g1_2.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e2g1_16.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e6g1_11.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e3g1_10.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e2g1_12.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e7g1_5.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e5g1_11.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e4g1_16.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e6g1_8.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e8g1_20.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e2g1_4.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e6g1_3.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e8g1_14.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e10g1_10.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e4g1_2.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e7g1_2.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e1g1_11.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e2g1_13.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e6g1_19.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e10g1_4.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e4g1_19.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e5g1_14.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e10g1_13.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e6g1_13.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e6g1_5.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e2g1_7.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e10g1_1.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e3g1_2.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e9g1_20.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e10g1_5.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e8g1_6.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e6g1_15.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e1g1_10.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e4g1_9.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e3g1_3.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e1g1_12.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e4g1_14.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e7g1_18.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e4g1_6.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e3g1_11.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e8g1_9.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e6g1_12.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e6g1_2.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e5g1_4.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e4g1_17.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e6g1_6.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e8g1_2.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e9g1_3.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e6g1_10.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e10g1_19.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e1g1_6.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e7g1_8.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e4g1_7.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e3g1_1.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e9g1_7.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e8g1_10.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e2g1_9.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e2g1_11.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e2g1_6.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e9g1_17.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e8g1_7.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e1g1_4.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e7g1_3.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e1g1_2.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e6g1_7.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e2g1_19.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e2g1_17.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e1g1_3.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e5g1_3.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e8g1_1.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e5g1_18.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e1g1_13.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e9g1_5.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e7g1_1.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e4g1_20.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e3g1_7.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e10g1_20.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e3g1_6.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e7g1_9.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e5g1_15.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e5g1_8.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e4g1_5.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e2g1_2.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e8g1_19.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e9g1_1.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e8g1_16.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e8g1_12.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e8g1_11.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e10g1_9.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e9g1_12.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e10g1_7.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e8g1_8.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e8g1_18.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e7g1_6.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e2g1_14.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e3g1_16.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e10g1_15.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e4g1_1.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e8g1_17.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e10g1_18.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e3g1_17.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e2g1_18.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e7g1_13.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e10g1_3.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e4g1_15.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e4g1_8.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e10g1_11.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e2g1_20.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e9g1_11.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e8g1_3.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e7g1_12.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e6g1_17.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e5g1_20.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e3g1_5.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e5g1_1.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e3g1_13.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e9g1_19.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e8g1_13.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e5g1_7.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e6g1_20.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e2g1_1.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e4g1_12.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e3g1_12.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e2g1_5.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e9g1_18.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e7g1_14.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e4g1_10.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e5g1_9.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e9g1_15.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e3g1_15.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e1g1_9.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e1g1_5.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e2g1_15.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e7g1_20.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e5g1_19.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e3g1_9.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e1g1_8.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e6g1_18.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e7g1_11.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e5g1_5.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e2g1_3.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e9g1_14.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e9g1_2.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e5g1_16.csv',
                  'balanced_normalized_dataset/las_thetag_dg_w_e1g1_20.csv', 
                  'balanced_normalized_dataset/las_thetag_dg_w_e1g1_1.csv', 
                  'balanced_normalized_dataset/las_thetag_dg_w_e4g1_18.csv', 
                  'balanced_normalized_dataset/las_thetag_dg_w_e3g1_20.csv']

In [0]:
test_csv_list =  ['balanced_normalized_dataset/las_thetag_dg_w_e10g1_16.csv', 'balanced_normalized_dataset/las_thetag_dg_w_e7g1_15.csv', 'balanced_normalized_dataset/las_thetag_dg_w_e7g1_16.csv', 'balanced_normalized_dataset/las_thetag_dg_w_e6g1_4.csv', 'balanced_normalized_dataset/las_thetag_dg_w_e6g1_16.csv', 'balanced_normalized_dataset/las_thetag_dg_w_e1g1_18.csv', 'balanced_normalized_dataset/las_thetag_dg_w_e5g1_17.csv', 'balanced_normalized_dataset/las_thetag_dg_w_e4g1_4.csv', 'balanced_normalized_dataset/las_thetag_dg_w_e5g1_12.csv', 'balanced_normalized_dataset/las_thetag_dg_w_e1g1_16.csv', 'balanced_normalized_dataset/las_thetag_dg_w_e4g1_3.csv', 'balanced_normalized_dataset/las_thetag_dg_w_e1g1_17.csv', 'balanced_normalized_dataset/las_thetag_dg_w_e10g1_8.csv', 'balanced_normalized_dataset/las_thetag_dg_w_e5g1_10.csv', 'balanced_normalized_dataset/las_thetag_dg_w_e10g1_12.csv', 'balanced_normalized_dataset/las_thetag_dg_w_e8g1_4.csv', 'balanced_normalized_dataset/las_thetag_dg_w_e10g1_6.csv', 'balanced_normalized_dataset/las_thetag_dg_w_e1g1_19.csv', 'balanced_normalized_dataset/las_thetag_dg_w_e7g1_17.csv', 'balanced_normalized_dataset/las_thetag_dg_w_e3g1_14.csv', 'balanced_normalized_dataset/las_thetag_dg_w_e9g1_8.csv', 'balanced_normalized_dataset/las_thetag_dg_w_e2g1_10.csv', 'balanced_normalized_dataset/las_thetag_dg_w_e9g1_4.csv', 'balanced_normalized_dataset/las_thetag_dg_w_e7g1_10.csv', 'balanced_normalized_dataset/las_thetag_dg_w_e3g1_19.csv', 'balanced_normalized_dataset/las_thetag_dg_w_e3g1_18.csv', 'balanced_normalized_dataset/las_thetag_dg_w_e5g1_13.csv', 'balanced_normalized_dataset/las_thetag_dg_w_e10g1_17.csv', 'balanced_normalized_dataset/las_thetag_dg_w_e6g1_14.csv', 'balanced_normalized_dataset/las_thetag_dg_w_e8g1_15.csv', 'balanced_normalized_dataset/las_thetag_dg_w_e7g1_19.csv', 'balanced_normalized_dataset/las_thetag_dg_w_e4g1_11.csv', 'balanced_normalized_dataset/las_thetag_dg_w_e9g1_9.csv', 'balanced_normalized_dataset/las_thetag_dg_w_e1g1_7.csv', 'balanced_normalized_dataset/las_thetag_dg_w_e8g1_5.csv', 'balanced_normalized_dataset/las_thetag_dg_w_e9g1_13.csv', 'balanced_normalized_dataset/las_thetag_dg_w_e3g1_4.csv', 'balanced_normalized_dataset/las_thetag_dg_w_e6g1_9.csv', 'balanced_normalized_dataset/las_thetag_dg_w_e5g1_6.csv', 'balanced_normalized_dataset/las_thetag_dg_w_e1g1_14.csv', 'balanced_normalized_dataset/las_thetag_dg_w_e2g1_8.csv', 'balanced_normalized_dataset/las_thetag_dg_w_e7g1_4.csv', 'balanced_normalized_dataset/las_thetag_dg_w_e9g1_16.csv', 'balanced_normalized_dataset/las_thetag_dg_w_e6g1_1.csv', 'balanced_normalized_dataset/las_thetag_dg_w_e9g1_10.csv', 'balanced_normalized_dataset/las_thetag_dg_w_e3g1_8.csv', 'balanced_normalized_dataset/las_thetag_dg_w_e4g1_13.csv', 'balanced_normalized_dataset/las_thetag_dg_w_e7g1_7.csv', 'balanced_normalized_dataset/las_thetag_dg_w_e1g1_15.csv', 'balanced_normalized_dataset/las_thetag_dg_w_e5g1_2.csv']

In [97]:
print('No. of training CSV files:', len(train_csv_list))
print('No. of test CSV files:', len(test_csv_list))

No. of training CSV files: 148
No. of test CSV files: 50


In [0]:
max_epoch = 100
num_classes = 3

In [0]:
model = MyModel(input_shape_X = (1020, 1), input_shape_Goal = (2, ))
model.compile(optimizer = 'sgd', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [100]:
print("Train on {} CSV files" .format(len(train_csv_list)))
best_val_acc = 0
for epoch in range(max_epoch):
  print('Epoch %d/%d' % (epoch + 1, max_epoch))
  epoch_loss = []
  epoch_acc = []
  i = 1
  tic = time.time()
  for CSV in train_csv_list:
    df = pd.read_csv(CSV, header = 0)

    x_train = df.iloc[:, 1: -1] # dropping 1st and last column
    y_train = df.iloc[:, -1:]

    x_train_goal_information = x_train.iloc[:, -2:]
    x_train = x_train.iloc[:, :-2]

    # Changing to 3-dim for Conv1D
    x_train = np.expand_dims(x_train, axis = 2)
    #print ("x_train shape: " + str(x_train.shape))
    
    # Scaling it up to positive integers (for applying to_categorical())
    y_train = y_train * 2
    
    # Converting class vectors to binary class matrices
    y_train = utils.to_categorical(y_train, num_classes)
    
    [loss, acc] = model.train_on_batch([x_train, x_train_goal_information], y_train)
    
    """
    acc: It's the average of the accuracy values for each batch of training data during training.
    
    loss: It is the average of training loss values calculated for each batch of training data during training. 
    Loss fucntion: Categorical Cross-Entropy.
    Optimizer: Stochastic Gradient Descent.
    """
    
    epoch_loss.append(loss)
    epoch_acc.append(acc)
    
    # info = 'Step: {0:d} - loss: {1:6.10f} - acc: {2:6.10f}'
    # print(info.format(i, loss, acc))
    
    i = i + 1
  toc = time.time()
  print('Elapsed time:', toc - tic, 'seconds |', 'epoch_loss:', sum(epoch_loss) / len(train_csv_list), '| epoch_acc:', sum(epoch_acc) / len(train_csv_list))
  
  """
  epoch_loss: The average of the loss of batches in that epoch.
  epoch_acc: The average of the accuracy of batches in that epoch.
  """
  
  total_loss = []
  total_acc = []
  i = 1
  for CSV in test_csv_list:
    df = pd.read_csv(CSV, header = 0)
    # df.head()

    x_test = df.iloc[:, 1: -1] # dropping 1st and last column
    y_test = df.iloc[:, -1:]

    x_test_goal_information = x_test.iloc[:, -2:]
    x_test = x_test.iloc[:, :-2]
    # x_train_goal_information.head()

    # Changing to 3-dim for Conv1D
    x_test = np.expand_dims(x_test, axis = 2)
    # print ("x_test shape: " + str(x_test.shape))

    # Scaling it up to positive integers
    y_test = y_test * 2

    # Converting class vectors to binary class matrices
    y_test = utils.to_categorical(y_test, num_classes)

    [loss2, acc2] = model.test_on_batch([x_test, x_test_goal_information], y_test)
    total_loss.append(loss2)
    total_acc.append(acc2)
    i = i + 1
    
    # Calculating the average
    val_acc = sum(total_acc) / len(test_csv_list)
  
  # Model Check-Point
  if((best_val_acc < val_acc) and (val_acc > 0.87)):
    name = 'epoch_' + str(epoch + 1) + '_val_acc_' + str(val_acc) + '.h5'
    model.save(name)
    best_val_acc = val_acc
      
  print('Elapsed time:', toc - tic, 'seconds | val_loss:', sum(total_loss) / len(test_csv_list), '| val_acc:', val_acc)
  print()

Train on 148 CSV files
Epoch 1/100
Elapsed time: 19.919479370117188 seconds | epoch_loss: 1.2546489883113552 | epoch_acc: 0.5055264222460825
Elapsed time: 19.919479370117188 seconds | val_loss: 0.7671912097930909 | val_acc: 0.6522101962566376

Epoch 2/100
Elapsed time: 16.70999574661255 seconds | epoch_loss: 0.8577435894592388 | epoch_acc: 0.5968932587955449
Elapsed time: 16.70999574661255 seconds | val_loss: 0.7045507967472077 | val_acc: 0.6998814713954925

Epoch 3/100
Elapsed time: 16.772029161453247 seconds | epoch_loss: 0.7694819223236393 | epoch_acc: 0.6377644599289507
Elapsed time: 16.772029161453247 seconds | val_loss: 0.6454787653684616 | val_acc: 0.7384855091571808

Epoch 4/100
Elapsed time: 16.596043348312378 seconds | epoch_loss: 0.7021969375578133 | epoch_acc: 0.678205193297283
Elapsed time: 16.596043348312378 seconds | val_loss: 0.5889392060041427 | val_acc: 0.747321925163269

Epoch 5/100
Elapsed time: 16.60897183418274 seconds | epoch_loss: 0.6606628335005528 | epoch_acc:

In [101]:
print("Test on {} CSV files" .format(len(test_csv_list)))
total_loss = []
total_acc = []
tic = time.time()
i = 1
for CSV in test_csv_list:
  df = pd.read_csv(CSV, header = 0)
  # df.head()

  x_test = df.iloc[:, 1: -1] # dropping 1st and last column
  y_test = df.iloc[:, -1:]

  x_test_goal_information = x_test.iloc[:, -2:]
  x_test = x_test.iloc[:, :-2]
  # x_train_goal_information.head()

  # Changing to 3-dim for Conv1D
  x_test = np.expand_dims(x_test, axis = 2)
  # print ("x_test shape: " + str(x_test.shape))

  # Scaling it up to positive integers
  y_test = y_test * 2

  # Converting class vectors to binary class matrices
  y_test = utils.to_categorical(y_test, num_classes)

  [loss2, acc2] = model.test_on_batch([x_test, x_test_goal_information], y_test)
  total_loss.append(loss2)
  total_acc.append(acc2)
  """
  acc2: It's the average of the accuracy values for each batch of test data during testing.
  
  loss2: It is the average of training loss values calculated for each batch of test data during testing. 
  Loss fucntion: Categorical Cross-Entropy.
  """
  print('CSV file: {} - test_loss: {} - test_acc: {}'.format(i, loss2, acc2))
  i = i + 1
toc = time.time()
print('Elapsed time:', toc - tic, 'seconds | total_test_loss:', sum(total_loss) / len(test_csv_list), ' total_test_acc:', sum(total_acc) / len(test_csv_list))

Test on 50 CSV files
CSV file: 1 - test_loss: 0.17856895923614502 - test_acc: 0.9223300814628601
CSV file: 2 - test_loss: 1.026901125907898 - test_acc: 0.7884615659713745
CSV file: 3 - test_loss: 0.7265350818634033 - test_acc: 0.7962962985038757
CSV file: 4 - test_loss: 0.0365322045981884 - test_acc: 1.0
CSV file: 5 - test_loss: 0.049343984574079514 - test_acc: 0.9912280440330505
CSV file: 6 - test_loss: 0.3403328061103821 - test_acc: 0.93388432264328
CSV file: 7 - test_loss: 0.7526093125343323 - test_acc: 0.8873239159584045
CSV file: 8 - test_loss: 0.38410264253616333 - test_acc: 0.8928571343421936
CSV file: 9 - test_loss: 0.5035429000854492 - test_acc: 0.8468468189239502
CSV file: 10 - test_loss: 0.6147133111953735 - test_acc: 0.9007633328437805
CSV file: 11 - test_loss: 0.9819380640983582 - test_acc: 0.7692307829856873
CSV file: 12 - test_loss: 0.6057435870170593 - test_acc: 0.9008264541625977
CSV file: 13 - test_loss: 0.13980966806411743 - test_acc: 0.8991596698760986
CSV file: 14 